##### __Step 1__ - To run this code, the following python packages must be downloaded: sys, pymatgen, numpy, pandas, matplotlib, math, chemparse, and os
##### __Step 2__ - Go to https://materialsproject.org and generate an API key. Label your generated key below as a string named MAPI_KEY

In [2]:
MAPI_KEY = '5Q7bbBN2ofCcBiIjbErlbTzRrZ7WU3O1'

##### __Step 3__ - Import the python module oxide_matching, which contains the functions necessary for this code.

In [3]:
import oxide_matching as om
import math
import pandas as pd

/Users/clairewu/opt/anaconda3/envs/lattice_matching_oxides/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### __Step 4__ - __Download all possible cubic oxides.__
##### _4.1_ - List elements to exclude in analysis 
##### _4.2_ - The data will save as a pandas dataframe. Choose a filename.
##### _4.3_ - Download all possible cubic oxides. The data will save as pandas dataframe, which will be used in step 5. This may take 10-15 mins and only needs to be done once.

In [4]:
#4.1 
elements_to_avoid = ['H', 'Be', 'Os', 'Hg', 'P', 'F','Cl','Br','I', 'At', 'Ts', 'Na', 'Li', 'K', 'Rb','Cs','Ac','Th','Pa','U','Np','Pu','Am','Cm','Bk','Cf','Es','Fm','Md','No','Lr', 'Na']

#4.2
file_name = 'oxide_data'

#4.3
om.download_oxides_from_matproj(MAPI_KEY, elements_to_avoid, file_name)

Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 5405.03it/s]


##### __Step 5__ - __Find lattice matched films to a particular substrate.__
##### _5.1_ - Specifiy the substrate,CTE (coefficient of thermal expansion), substrate orientation, xtal_type, and lattice parameter.
##### _5.2_ - Specify minimum % strain
##### _5.3_ - Specify file_name that corresponds to the oxide data (same as step 4.3)
##### _5.4_ - Find lattice matched films. The output is a pandas dataframe that contains all unique matches for the (100), (110), and (111) MgO substrates. 

In [12]:
#5.1
substrate = 'MgO'
a = 4.216
CTE = 10.8

sub_xtal_type = 'rocksalt'
sub_orientation = ['(100)', '(110)', '(111)']
lattice_parameter = [a, a*math.sqrt(2), a*math.sqrt(3)]

# 5.2 
strain = 5

# 5.3
file_name = 'oxide_data'

# 5.4
old_films = pd.DataFrame([])
for ii in range(0, len(sub_orientation)):
    films = om.find_films(file_name, substrate, sub_xtal_type,lattice_parameter[ii], sub_orientation[ii], strain, CTE)
    films['sub_orientation'] = sub_orientation[ii]
    films['substrate'] = substrate
    old_films = pd.concat([old_films, films])
    
films = old_films.drop_duplicates(subset=['film'])
films = films.reset_index(drop=True)
films.to_csv(r'films_on_'+substrate+'.csv', index = False, header=True)

print('There are '+str(len(films[(films['stability']== 'stable')]))+' stable films predicted')
print('There are '+str(len(films[~(films['stability']== 'stable')]))+' metastable or potentially unstable films predicted')
print('We label x-tal types based on the stoichometry of the oxides. This may result in some films be inaccurately labeled')

#pd.options.display.max_columns = 1000
#pd.options.display.max_rows = 1000
#display(films)

There are 49 stable films predicted
There are 287 metastable or potentially unstable films predicted
We label x-tal types based on the stoichometry of the oxides. This may result in some films be inaccurately labeled


##### _Optional_ - __Narrow list to stable films with a non-zero refractive index and bandgap larger than 1.75 eV.__

In [13]:
films = films[(films['stability'] == 'stable') &
                (films['n'] > 0) & 
                (films['band gap (eV)'] > 2)]
 
#pd.options.display.max_columns = 1000
#pd.options.display.max_rows = 1000
display(films)

,film,ICSD?,band gap (eV),n,stability,B,mu,nsites,elements,MW,density,CTE,x-tal,a,film orientation,film lattice parameter,% strain,sub_orientation,substrate
2,BaZrO3,true,3.0377,2.220,stable,148.867334,0.244305,5.0,"[Ba, Zr, O]",276.548,6.143794,9.138680,perovskite,4.212,(100),4.2120,0.094877,(100),MgO
17,MgCr2O4,true,2.5264,2.106,stable,0.000000,0.000000,7.0,"[Mg, Cr, O]",192.293,4.371846,0.000000,spinel,8.360,(100)/2,4.1800,0.853890,(100),MgO
24,Zn(GaO2)2,true,2.3105,2.054,stable,168.530793,0.300462,7.0,"[O, Ga, Zn]",268.832,6.151539,14.031353,spinel,8.342,(100)/2,4.1710,1.067362,(100),MgO
42,Ni2GeO4,true,2.1356,2.100,stable,0.000000,0.000000,7.0,"[Ni, Ge, O]",254.022,6.073625,0.000000,spinel,8.221,(100)/2,4.1105,2.502372,(100),MgO
48,MnAl2O4,true,2.6578,1.865,stable,180.219678,0.310633,7.0,"[Mn, Al, O]",172.898,4.186128,14.391688,spinel,8.187,(100)/2,4.0935,2.905598,(100),MgO
69,MgAl2O4,true,5.1095,1.755,stable,180.375591,0.274363,7.0,"[Mg, Al, O]",142.265,3.582136,13.195893,spinel,8.080,(100)/2,4.0400,4.174573,(100),MgO
71,Al2ZnO4,true,3.8460,1.857,stable,0.000000,0.000000,7.0,"[Al, Zn, O]",183.350,4.625889,0.000000,spinel,8.075,(100)/2,4.0375,4.233871,(100),MgO
121,BaHfO3,false,3.5394,2.156,stable,159.021544,0.234462,5.0,"[Ba, Hf, O]",363.814,8.332451,8.550653,perovskite,4.170,(100),4.1700,1.091082,(100),MgO
334,BaLa2O4,false,3.7033,1.991,stable,0.000000,0.000000,7.0,"[Ba, La, O]",479.135,5.497951,0.000000,spinel,7.412,(100),7.4120,1.501902,(111),MgO


In [ ]:
# other potential substrate candidates
 
substrate = 'YZrO2'
a = 5.125
sub_xtal_type = 'fluorite'

substrate = 'SrTiO3'
sub_xtal_type = 'perovskite'    
a = 3.905

substrate = 'LaAlO3'     
sub_xtal_type = 'perovskite'    
a = 3.792

substrate = 'MgAl2O4'
a = 8.083
sub_xtal_type = 'spinel'

substrate = 'CdO'
a = 4.6958
sub_xtal_type = 'rocksalt'

